In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set project root directory
PROJECT_ROOT = '/mnt/c/Users/Linda/Desktop/GitHub_Projects/MLP_ProteinLocalization'

# Load data
features = pd.read_csv(os.path.join(PROJECT_ROOT, 'data', 'features.csv'))
labels = pd.read_csv(os.path.join(PROJECT_ROOT, 'data', 'labels.csv'))

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels['Type'])

# Scale features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels_encoded, test_size=0.2, random_state=42)

# Function to evaluate model
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    sensitivity = np.diag(cm) / np.sum(cm, axis=1)
    specificity = np.diag(cm) / np.sum(cm, axis=0)
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, sensitivity, specificity, f1, cm

# Function to plot confusion matrix
def plot_confusion_matrix(cm, class_names, title):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    os.makedirs(os.path.join(PROJECT_ROOT, 'results'), exist_ok=True)
    plt.savefig(os.path.join(PROJECT_ROOT, 'results', f"{title.lower().replace(' ', '_')}.png"))
    plt.close()

# Function to train and evaluate model
def train_and_evaluate(X, y, model_name, sampling_method=None):
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    cv_scores = {'accuracy': [], 'sensitivity': [], 'specificity': [], 'f1': []}
    
    for fold, (train_index, val_index) in enumerate(skf.split(X, y), 1):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]
        
        if sampling_method == 'oversample':
            sampler = SMOTE(random_state=42)
            X_train_fold, y_train_fold = sampler.fit_resample(X_train_fold, y_train_fold)
        elif sampling_method == 'undersample':
            sampler = RandomUnderSampler(random_state=42)
            X_train_fold, y_train_fold = sampler.fit_resample(X_train_fold, y_train_fold)
        
        model = MLPClassifier(hidden_layer_sizes=(256, 128, 64), activation='relu', solver='adam', alpha=0.001, learning_rate='adaptive', max_iter=1000, batch_size=64, early_stopping=True, validation_fraction=0.2, random_state=42)
        model.fit(X_train_fold, y_train_fold)
        
        y_pred = model.predict(X_val_fold)
        accuracy, sensitivity, specificity, f1, _ = evaluate_model(y_val_fold, y_pred)
        cv_scores['accuracy'].append(accuracy)
        cv_scores['sensitivity'].append(np.mean(sensitivity))
        cv_scores['specificity'].append(np.mean(specificity))
        cv_scores['f1'].append(f1)
        
        print(f"\n{model_name} - Fold {fold} Results:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Sensitivity: {np.mean(sensitivity):.4f}")
        print(f"Specificity: {np.mean(specificity):.4f}")
        print(f"F1 Score: {f1:.4f}")
    
    # Average scores across folds
    avg_scores = {metric: np.mean(cv_scores[metric]) for metric in cv_scores}
    print(f"\n{model_name} - Average Cross-Validation Results:")
    for metric, avg in avg_scores.items():
        print(f"{metric.capitalize()}: {avg:.4f}")
    return avg_scores

# Function to plot feature importance
def plot_feature_importance(model, X, y, feature_names, title):
    result = permutation_importance(model, X, y, n_repeats=10, random_state=42)
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': result.importances_mean
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
    plt.title(title)
    plt.tight_layout()
    plt.savefig(os.path.join(PROJECT_ROOT, 'results', f"{title.lower().replace(' ', '_')}.png"))
    plt.close()

# Test model function
def test_model(X_train, y_train, X_test, y_test, model_name, sampling_method=None):
    if sampling_method == 'oversample':
        sampler = SMOTE(random_state=42)
        X_train, y_train = sampler.fit_resample(X_train, y_train)
    elif sampling_method == 'undersample':
        sampler = RandomUnderSampler(random_state=42)
        X_train, y_train = sampler.fit_resample(X_train, y_train)
    
    model = MLPClassifier(hidden_layer_sizes=(256, 128, 64), activation='relu', solver='adam', alpha=0.001, learning_rate='adaptive', max_iter=1000, batch_size=64, early_stopping=True, validation_fraction=0.2, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy, sensitivity, specificity, f1, cm = evaluate_model(y_test, y_pred)
    
    print(f"\n{model_name} - Test Set Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Sensitivity: {np.mean(sensitivity):.4f}")
    print(f"Specificity: {np.mean(specificity):.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    plot_confusion_matrix(cm, le.classes_, f"{model_name} Test Set Confusion Matrix")
    plot_feature_importance(model, X_test, y_test, features.columns, f"{model_name} Feature Importance")
    
    return model

# Train and evaluate on imbalanced data
print("Training and evaluating imbalanced model:")
imbalanced_avg = train_and_evaluate(X_train, y_train, "Imbalanced MLP")

# Train and evaluate with oversampling
print("\nTraining and evaluating balanced model with oversampling:")
balanced_over_avg = train_and_evaluate(X_train, y_train, "Balanced MLP (Oversampling)", sampling_method='oversample')

# Train and evaluate with undersampling
print("\nTraining and evaluating balanced model with undersampling:")
balanced_under_avg = train_and_evaluate(X_train, y_train, "Balanced MLP (Undersampling)", sampling_method='undersample')

# Test each model
imbalanced_model = test_model(X_train, y_train, X_test, y_test, "Imbalanced MLP")
oversampled_model = test_model(X_train, y_train, X_test, y_test, "Balanced MLP (Oversampling)", sampling_method='oversample')
undersampled_model = test_model(X_train, y_train, X_test, y_test, "Balanced MLP (Undersampling)", sampling_method='undersample')

print("\nConfusion matrices for test results and feature importance plots have been saved in the results folder.")

Training and evaluating imbalanced model:

Imbalanced MLP - Fold 1 Results:
Accuracy: 0.9195
Sensitivity: 0.5841
Specificity: 0.6444
F1 Score: 0.9165


/tmp/ipykernel_1440/2667551215.py:37: RuntimeWarning: invalid value encountered in divide
  specificity = np.diag(cm) / np.sum(cm, axis=0)



Imbalanced MLP - Fold 2 Results:
Accuracy: 0.9343
Sensitivity: 0.6679
Specificity: nan
F1 Score: 0.9332

Imbalanced MLP - Fold 3 Results:
Accuracy: 0.9300
Sensitivity: 0.6534
Specificity: 0.7028
F1 Score: 0.9282

Imbalanced MLP - Fold 4 Results:
Accuracy: 0.9284
Sensitivity: 0.6524
Specificity: 0.6996
F1 Score: 0.9271

Imbalanced MLP - Average Cross-Validation Results:
Accuracy: 0.9281
Sensitivity: 0.6394
Specificity: nan
F1: 0.9263

Training and evaluating balanced model with oversampling:

Balanced MLP (Oversampling) - Fold 1 Results:
Accuracy: 0.9211
Sensitivity: 0.6501
Specificity: 0.6859
F1 Score: 0.9202
